In [1]:
from sgp4.api import Satrec
from sgp4 import exporter
import pykep as pk
import numpy as np

class tle:
    """__init__(line1, line2)

    This User Defined Planet (UDPLA) represents a satellite orbiting the Earth and defined in the TLE format
    and propagated using the SGP4 propagator.

    .. note::
       The resulting ephemerides will be returned in SI units and in the True Equator Mean Equinox (TEME) reference frame

    """
    def __init__(self, line1, line2):
        import pykep as pk
        self.satellite = Satrec.twoline2rv(line1, line2)
        self.e = 0
        self.ref_epoch = pk.epoch(self.satellite.jdsatepoch + self.satellite.jdsatepochF, pk.epoch.julian_type.JD)
    def eph(self, mjd2000):
        jd = mjd2000 + 2451544.5
        jd_i = int(jd)
        jd_fr = jd-jd_i
        self.e, r, v = self.satellite.sgp4(jd_i, jd_fr)
        return [[it*1000 for it in r], [it*1000 for it in v]]
    def eph_v(self, mjd2000s):
        jds = [mjd2000 + 51544 + 2451544.5 for mjd2000 in mjd2000s]
        jd_is = [int(item) for item in jds]
        jd_frs = [a-b for a,b in zip(jds,jd_is)]
        self.e, r, v = self.satellite.sgp4_array(np.array(jd_is), np.array(jd_frs))
        rv = np.hstack((r,v))
        return rv.reshape((-1, 6)) 
    def get_name(self):
        return self.satellite.satnum_str + " - SGP4"
    def get_extra_info(self):
        line1, line2 = exporter.export_tle(self.satellite)
        return "TLE line1: " + line1 + "\nTLE line2: " + line2 
    def get_mu_central_body(self):
        return pk.MU_EARTH

In [2]:
line1 = '1 25544U 98067A   19343.69339541  .00001764  00000-0  38792-4 0  9991'
line2 = '2 25544  51.6439 211.2001 0007417  17.6667  85.6398 15.50103472202482'
udpla = tle(line1, line2)
mjd2000s = np.array([when%1000.2 for when in range(20000)])
satellite = Satrec.twoline2rv(line1, line2)


In [3]:
pla = pk.planet(udpla)

In [4]:
%%timeit
for mjd2000 in mjd2000s:
    r = udpla.eph(mjd2000)

18.8 ms ± 202 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%%timeit
for mjd2000 in mjd2000s:
    r = pla.eph(mjd2000)

45.8 ms ± 602 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
r = pla.eph_v(mjd2000s)

8.04 ms ± 155 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%%timeit
for mjd2000 in mjd2000s:
    jd = mjd2000 + 2451544.5
    jd_i = int(jd)
    jd_fr = jd-jd_i
    e, r, v = satellite.sgp4(jd_i, jd_fr)


7.48 ms ± 135 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit
jds = [mjd2000 + 2451544.5 for mjd2000 in mjd2000s]
jd_is = [int(item) for item in jds]
jd_frs = [a-b for a,b in zip(jds, jd_is)]
e, r, v = satellite.sgp4_array(np.array(jd_is), np.array(jd_frs))


5.4 ms ± 74.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
import spiceypy as pyspice


In [10]:
pk.utils.load_spice_kernels("../../pykep/data/de440s.bsp")

In [11]:
rv, _ = pyspice.spkezr("JUPITER BARYCENTER", [0.,1.,2.,3], "ECLIPJ2000", "NONE", "SSB")

In [12]:
np.array(rv).shape

(4, 6)

In [13]:
udpla = pk.udpla.spice("JUPITER BARYCENTER", "ECLIPJ2000", "SSB")


In [14]:
pla = pk.planet(udpla)

In [17]:
%%timeit
for mjd2000 in mjd2000s:
    udpla.eph(mjd2000)

115 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
for mjd2000 in mjd2000s:
    r = pla.eph(mjd2000)

155 ms ± 320 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%%timeit
r = pla.eph_v(mjd2000s)

46 ms ± 544 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%%timeit
for mjd2000 in mjd2000s:
    rv, _ = pyspice.spkezr("JUPITER BARYCENTER", (mjd2000-0.5)*pk.DAY2SEC, "ECLIPJ2000", "NONE", "SSB")

71.4 ms ± 355 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%%timeit
for mjd2000 in mjd2000s:
    rv, _ = pyspice.spkezr("JUPITER BARYCENTER", (mjd2000-0.5)*pk.DAY2SEC), "ECLIPJ2000", "NONE", "SSB")

KeyboardInterrupt: 